In [1]:
# | default_exp client

# Client

> base classes and functions for this library

In [2]:
# | exporti
from typing import Any, Union
from dataclasses import dataclass, field
from abc import abstractmethod, ABC

import httpx
import json

In [3]:
# | hide
from nbdev.showdoc import show_doc

In [4]:
# | export


@dataclass
class Auth(ABC):
    """Base class for authentication"""

    @abstractmethod
    def get_auth_headers(self) -> dict:
        """Get the headers for the authentication"""
        pass

# Response Classes

In [5]:
# | export
@dataclass
class ResponseGetData:
    is_from_cache: bool
    is_success: bool
    status: int
    response: Any
    auth: Any = field(repr=False, default=None)

    @classmethod
    def _from_httpx(cls, response: httpx.Response, auth: Any = None):
        return cls(
            status=response.status_code,
            response=response.json(),
            is_success=response.is_success,
            is_from_cache=False,
            auth=auth,
        )

    @classmethod
    def _from_cache(cls, data: dict = None, auth: Any = None):
        return cls(
            status=200,
            response=data,
            is_success=True,
            is_from_cache=True,
            auth=auth,
        )

In [6]:
show_doc(ResponseGetData._from_httpx)

---

### ResponseGetData._from_httpx

>      ResponseGetData._from_httpx (response:httpx.Response, auth:Any=None)

# Cache Handlers

In [7]:
# | export
def get_cache(
    json_cache_path: str,
    debug_api: bool = False,
) -> Union[dict, None]:
    """function for getting cached data from json file"""

    json_data = None
    try:
        with open(json_cache_path, "r", encoding="utf-8") as file:
            json_data = json.load(file)

    except (FileNotFoundError, json.JSONDecodeError) as e:
        json_data = None

    if json_data:
        if debug_api:
            print(f"🚀 Using cached data in {json_cache_path}")

    return json_data


def update_cache(json_cache_path: str, json_data: dict):
    with open(json_cache_path, "w", encoding="utf-8") as file:
        json.dump(json_data, file)

    return True

In [8]:
show_doc(get_cache)

---

### get_cache

>      get_cache (json_cache_path:str, debug_api:bool=False)

function for getting cached data from json file

In [9]:
test_cache_path = "../TEST/cache.json"

assert test_cache_path

get_cache(test_cache_path)

{'cases_time_series': [{'dailyconfirmed': '1',
   'dailydeceased': '0',
   'dailyrecovered': '0',
   'date': '30 January 2020',
   'dateymd': '2020-01-30',
   'totalconfirmed': '1',
   'totaldeceased': '0',
   'totalrecovered': '0'},
  {'dailyconfirmed': '0',
   'dailydeceased': '0',
   'dailyrecovered': '0',
   'date': '31 January 2020',
   'dateymd': '2020-01-31',
   'totalconfirmed': '1',
   'totaldeceased': '0',
   'totalrecovered': '0'},
  {'dailyconfirmed': '0',
   'dailydeceased': '0',
   'dailyrecovered': '0',
   'date': '1 February 2020',
   'dateymd': '2020-02-01',
   'totalconfirmed': '1',
   'totaldeceased': '0',
   'totalrecovered': '0'},
  {'dailyconfirmed': '1',
   'dailydeceased': '0',
   'dailyrecovered': '0',
   'date': '2 February 2020',
   'dateymd': '2020-02-02',
   'totalconfirmed': '2',
   'totaldeceased': '0',
   'totalrecovered': '0'},
  {'dailyconfirmed': '1',
   'dailydeceased': '0',
   'dailyrecovered': '0',
   'date': '3 February 2020',
   'dateymd': '2020-

# Get Data

In [10]:
# | exporti
def prepare_fetch(
    url: str,
    params: dict = None,
    auth: Auth = None,
    headers: dict = None,
    body: dict = None,
):
    """base function to prepare a fetch operation"""

    headers = headers or {}

    if auth:
        headers = {**auth.get_auth_headers(), **headers}

    return headers, url, params, body

In [11]:
# |export
async def get_data(
    url: str,
    method: str,
    client: httpx.AsyncClient,
    json_cache_path: str,
    is_ignore_cache: bool = False,
    headers: dict = None,
    params: dict = None,
    body: dict = None,
    auth: any = None,
    parent_class: str = None,
    debug_api: bool = False,
) -> Any:
    if not is_ignore_cache:
        json_data = get_cache(json_cache_path=json_cache_path, debug_api=debug_api)

        if json_data:
            return ResponseGetData._from_cache(data=json_data, auth=auth)

    headers, url, params, body = prepare_fetch(
        url=url,
        params=params,
        auth=auth,
        headers=headers,
        body=body,
    )

    if debug_api:
        print(
            {
                "headers": headers,
                "url": url,
                "params": params,
                "body": body,
                "cache_file_path": json_cache_path,
                "debug_api": debug_api,
                "parent_class": parent_class,
            }
        )

    client = client or httpx.AsyncClient()

    if method.upper() == "GET":
        res = await client.get(
            url=url, headers=headers, params=params, follow_redirects=True
        )

    rgd = ResponseGetData._from_httpx(res, auth=auth)

    if rgd.is_success:
        update_cache(json_cache_path=json_cache_path, json_data=rgd.response)

    return rgd

In [12]:
show_doc(get_data)

---

### get_data

>      get_data (url:str, method:str, client:httpx.AsyncClient,
>                json_cache_path:str, is_ignore_cache:bool=False,
>                headers:dict=None, params:dict=None, body:dict=None,
>                auth:<built-infunctionany>=None, parent_class:str=None,
>                debug_api:bool=False)

In [13]:
url = "https://api.covid19india.org/data.json"

await get_data(
    json_cache_path="../TEST/cache.json",
    url=url,
    method="GET",
    client=None,
    # is_ignore_cache=False,
    debug_api=False,
)

ResponseGetData(is_from_cache=True, is_success=True, status=200, response={'cases_time_series': [{'dailyconfirmed': '1', 'dailydeceased': '0', 'dailyrecovered': '0', 'date': '30 January 2020', 'dateymd': '2020-01-30', 'totalconfirmed': '1', 'totaldeceased': '0', 'totalrecovered': '0'}, {'dailyconfirmed': '0', 'dailydeceased': '0', 'dailyrecovered': '0', 'date': '31 January 2020', 'dateymd': '2020-01-31', 'totalconfirmed': '1', 'totaldeceased': '0', 'totalrecovered': '0'}, {'dailyconfirmed': '0', 'dailydeceased': '0', 'dailyrecovered': '0', 'date': '1 February 2020', 'dateymd': '2020-02-01', 'totalconfirmed': '1', 'totaldeceased': '0', 'totalrecovered': '0'}, {'dailyconfirmed': '1', 'dailydeceased': '0', 'dailyrecovered': '0', 'date': '2 February 2020', 'dateymd': '2020-02-02', 'totalconfirmed': '2', 'totaldeceased': '0', 'totalrecovered': '0'}, {'dailyconfirmed': '1', 'dailydeceased': '0', 'dailyrecovered': '0', 'date': '3 February 2020', 'dateymd': '2020-02-03', 'totalconfirmed': '3',

In [14]:
# |export


# async def fetch_post(client: httpx.AsyncClient):
#     data_to_post = {"key": "value"}
#     response = await client.post(f"{BASE_URL}/post", json=data_to_post)
#     return response.json()


# async def fetch_put(client: httpx.AsyncClient):
#     data_to_put = {"key": "updated_value"}
#     response = await client.put(f"{BASE_URL}/put", json=data_to_put)
#     return response.json()


# async def fetch_delete(client: httpx.AsyncClient):
#     response = await client.delete(f"{BASE_URL}/delete")
#     return response.json()

In [15]:
# | hide
import nbdev

nbdev.nbdev_export()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)